# Wissensaggregator Mittelalter und frühe Neuzeit

## Daten für Ämter von Domherren einlesen

Übertrage Daten aus Access. Die Daten werden aus Access via ODBC in eine Transfer-Datenbank geschrieben: 
`domherr.tbl_domherren` und `domherr.tbl_domherren_aemter`

[Amtsperiode](#Amtsperiode)  
[Zeitspanne der Person](#Zeitspanne-der-Person)  
[Archidiakonat](#Archidiakonat)  
[Sortierung der Person nach Amt](#Sortierung-der-Person-nach-Amt) (hinfällig?)

Vorspann: Pakete laden; mit der Datenbank verbinden

In [1]:
wds_path="../.."

"../.."

In [2]:
cd(wds_path)

In [3]:
using Pkg

In [4]:
Pkg.activate(".")

  Activating project at `C:\Users\georg\Documents\projekte\WiagDataSetup.jl`


In [5]:
using Revise, MySQL

In [6]:
using WiagDataSetup

┌ Info: Precompiling WiagDataSetup [522c5ebb-a018-4020-8ed4-420cb1a9f084]
└ @ Base loading.jl:1423


In [7]:
using MySQL, DataFrames, CSV

In [8]:
Wds = WiagDataSetup

WiagDataSetup

In [9]:
Wds.setDBWIAG(user="georg", db="wiag2")

Passwort für User georg: ········


MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2")

In [10]:
item_type_id = 5

5

Die Ämter in Tabelle `role` werden im Notebook "Strukturdaten" eingelesen.

## Amtsperiode

In [11]:
sql = "SELECT * FROM domherr.tbl_domherren_aemter"
df_pr = Wds.sql_df(sql);

In [12]:
Wds.clean_up!(df_pr);

Ignoriere Einträge ohne Amtsart/ID_Amt_norm

In [13]:
size(df_pr)

(33189, 20)

In [14]:
one_valid(a, b) = !ismissing(a) | !ismissing(b)

one_valid (generic function with 1 method)

In [15]:
subset!(df_pr, [:Amtsart, :ID_Amt_norm] => ByRow(one_valid));

In [16]:
df_pr[100:105, [:ID_Amt, :Amtsart, :ID_Amt_norm, :Amtsbeginn, :Amtsende]]

,ID_Amt,Amtsart,ID_Amt_norm,Amtsbeginn,Amtsende
,Int32,String?,Int32?,String?,String?
1,82183,missing,19,1324,missing
2,82184,missing,19,1324,1332
3,82185,missing,19,1324,missing
4,82186,missing,19,1290,1345
5,82187,missing,19,1325,1344
6,82188,missing,5,missing,missing


### Domherren zuordnen

In [17]:
df_pr[!, :id_in_source] .= string.(df_pr[!, :ID_Domherr]);

In [18]:
table_name = "item";
sql = "SELECT id AS person_id, id_in_source FROM $(table_name) WHERE item_type_id = $(item_type_id)"
df_idx = Wds.sql_df(sql);

In [19]:
df_prp = leftjoin(df_pr, df_idx, on = :id_in_source);

Berücksichtige Einträge nicht, wenn sie weder `Amtsart` noch `ID_Amt_norm` enthalten

In [20]:
one_valid(x, y) = !ismissing(x) || !ismissing(y)

one_valid (generic function with 1 method)

In [21]:
subset!(df_prp, [:Amtsart, :ID_Amt_norm] => ByRow(one_valid));

In [22]:
size(df_prp)

(32988, 22)

In [23]:
df_idx_mg = filter(:person_id => ismissing, df_prp);

In [24]:
size(df_idx_mg)

(0, 22)

### Rollen/Ämter zuordnen

In [25]:
sql = "SELECT name as role_name, role.id as role_id, item.id_in_source as role_id_in_source
FROM role JOIN item ON item.id = role.id"
df_role = Wds.sql_df(sql);

Werden alle Ämter gefunden?

In [26]:
role_dh = unique(string.(df_prp.ID_Amt_norm));

In [27]:
match_mg = setdiff(role_dh, df_role.role_id_in_source)

1-element Vector{String}:
 "missing"

In [28]:
to_string_maybe(a) = ismissing(a) ? a : string(a)

to_string_maybe (generic function with 1 method)

In [29]:
transform!(df_prp, :ID_Amt_norm => ByRow(to_string_maybe) => :match_role_id);

In [30]:
df_prp_r = leftjoin(df_prp, df_role, on = :match_role_id => :role_id_in_source, matchmissing = :notequal);

In [31]:
count(ismissing, df_prp_r.role_id), count(ismissing, df_prp_r.Amtsart)

(1216, 891)

*2022-03-16* TODO übernimm die Rollenbezeichnung `person_role.role_name` aus der Tabelle `role`.

### Bistümer zuordnen

In [32]:
sql = "SELECT id AS diocese_id, name AS diocese_name FROM diocese"
df_dioc = Wds.sql_df(sql);

Es sind nicht alle Ämter mit einem Bistum verknüpft, daher ist eine JOIN-Operation auf diesem Feld nicht möglich.

In [33]:
lookup_dioc = Dict(df_dioc[!,:diocese_name] .=> df_dioc[!,:diocese_id]);

In [34]:
lookup_dioc["Trier"]

30

In [35]:
get_dioc_id(name) = get(lookup_dioc, name, missing)

get_dioc_id (generic function with 1 method)

In [36]:
transform!(df_prp_r, :Bistum => ByRow(get_dioc_id) => :diocese_id);

In [37]:
idx_diocese = .!ismissing.(df_prp_r[!, :diocese_id]);

In [38]:
sum(idx_diocese)

913

### Instutionen zuordnen
Klöster, Domstifte

In [39]:
table_name = "institution";
sql = "SELECT id AS institution_id, name, id_gsn FROM $(table_name)"
df_inst = Wds.sql_df(sql);

In [40]:
lookup_inst = Dict(df_inst.id_gsn .=> df_inst.institution_id);

In [41]:
length(lookup_inst)

5286

In [42]:
get_inst_id(id_gsn) = get(lookup_inst, id_gsn, missing)

get_inst_id (generic function with 1 method)

In [43]:
get_inst_id(318)

79619

In [44]:
transform!(df_prp_r, :ID_Kloster => ByRow(get_inst_id) => :institution_id);

In [45]:
lookup_inst_name = Dict(df_inst.id_gsn .=> df_inst.name);

In [46]:
get_inst_name(id_gsn) = get(lookup_inst_name, id_gsn, missing)

get_inst_name (generic function with 1 method)

In [47]:
get_inst_name(318)

"Benediktinerinnenkloster Neuenwalde"

In [48]:
transform!(df_prp_r, :ID_Kloster => ByRow(get_inst_name) => :institution_name);

In [49]:
df_mg_inst_id = subset(df_prp_r, :institution_id => ByRow(ismissing));

In [50]:
size(df_mg_inst_id, 1), size(df_prp_r, 1)

(1418, 32988)

### Numerische Daten eintragen

In [51]:
num_date_begin(s_date) = Wds.parsemaybe(s_date, :lower)
df_prp_r[!, :num_date_begin] .= num_date_begin.(df_prp_r[!, :Amtsbeginn]);

┌ Warning: Could only find year in 
│   s = vo 1147
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1117
┌ Warning: Could not parse 
│   s = unbekannt
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1117
┌ Warning: Could only find year in 
│   s = t 1567
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could only find year in 
│   s = zwischen 1101
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could only find year in 
│   s = Bewerbung 1451
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could only find year in 
│   s = [1068]
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.j

In [52]:
num_date_end(s_date) = Wds.parsemaybe(s_date, :upper)
df_prp_r[!, :num_date_end] .= num_date_end.(df_prp_r[!, :Amtsende]);

┌ Warning: Could only find year in 
│   s = res. 1399
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could only find year in 
│   s = t1529
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could only find year in 
│   s = t1547
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could only find year in 
│   s = und 1118
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could only find year in 
│   s = 1 1320 Aug. 22
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1112
┌ Warning: Could not parse 
│   s = 66
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1117
┌ Warning: Could not parse 
│   s = 86
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup

In [53]:
df_info = dropmissing(df_prp_r, :Amtsart);

In [54]:
size(df_info, 1)

32097

In [55]:
df_info[60:67, [:ID_Domherr, :Amtsart, :Amtsbeginn, :num_date_begin, :Amtsende, :num_date_end]]

,ID_Domherr,Amtsart,Amtsbeginn,num_date_begin,Amtsende,num_date_end
,Int32,String,String?,Int64?,String?,Int64?
1,44969,Dekan,1604,1604,1605,1605
2,44964,Dekan,1605,1605,1621,1621
3,44970,Dekan,1622,1622,1634,1634
4,45034,Dekan,1634,1634,1638,1638
5,45035,Dekan,1638,1638,1652,1652
6,44972,Dekan,1653,1653,1668,1668
7,44973,Dekan,1668,1668,1679,1679
8,45038,Dekan,1679,1679,1685,1685


In [56]:
size(df_prp)

(32988, 23)

In [57]:
transform!(df_prp_r, :Amtsbeginn => ByRow(Wds.parse_year_sort) => :date_sort_key);

┌ Warning: could not parse 
│   s = vo 1147
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: year out of range in 
│   s = um 12100
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1506
┌ Warning: year out of range in 
│   s = um 12115
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1506
┌ Warning: year out of range in 
│   s = 12239
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1506
┌ Warning: year out of range in 
│   s = 4563
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1506
┌ Warning: could not parse 
│   s = (1553) 1560
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = (1574) 1593
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ 

In [58]:
transform!(df_prp_r, :Amtsende => ByRow(Wds.parse_year_sort) => :date_sort_key_replace);

┌ Warning: year out of range in 
│   s = 16445
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1506
┌ Warning: could not parse 
│   s = res. 1399
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = t1529
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = t1547
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = †1259
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = †1259
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = †1275
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   

┌ Warning: could not parse 
│   s = † 1579
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1585
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1606
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1614
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1619
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = †1356
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1203
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s =

┌ Warning: could not parse 
│   s = † 1301
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1364
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1407
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1506
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = †1543
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514
┌ Warning: could not parse 
│   s = † 1571
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1514


In [59]:
replace_if_missing(a, b) = ismissing(a) ? b : a

replace_if_missing (generic function with 1 method)

In [60]:
transform!(df_prp_r, [:date_sort_key, :date_sort_key_replace] => ByRow(replace_if_missing) => :date_sort_key);

In [61]:
size(df_prp_r, 1), count(ismissing, df_prp_r.date_sort_key)

(32988, 0)

In [62]:
df_info = dropmissing(df_prp_r, :Amtsart);

In [63]:
df_info[100:105, [:Amtsart, :ID_Amt_norm, :role_id, :num_date_begin, :num_date_end, :date_sort_key]]

,Amtsart,ID_Amt_norm,role_id,num_date_begin,num_date_end,date_sort_key
,String,Int32?,Int32?,Int64?,Int64?,Int64
1,Kantor,66,74090,1695,1699,1695150
2,Kantor,66,74090,1699,1709,1699150
3,Kantor,66,74090,1710,1732,1710150
4,Kantor,66,74090,1732,1738,1732150
5,Kantor,66,74090,1738,1743,1738150
6,Kantor,66,74090,1743,1750,1743150


Einträge für diesen Item_Typ löschen

In [64]:
table_name = "person_role"
sql = "DELETE FROM $(table_name) WHERE person_id IN 
(SELECT id FROM item WHERE item_type_id = $(item_type_id))"
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role WHERE person_id IN \n(SELECT id FROM item WHERE item_type_id = 5)", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

Daten schreiben

Schreibe für alle Bistümer - auch für die identifizierten - den Namen des Bistums in die Tabelle.  
Schreibe für alle Institutionen - auch für die identifizieren - die Bezeichnung in die Tabelle.  
Schreibe für alle Ämter - auch für die identifizieren - die Bezeichnung in die Tabelle.

Erzeuge die IDs selbst, um sie für `person_role_property` zur Verfügung zu haben.

In [65]:
table_name = "person_role"
sql = "SELECT MAX(id) + 1 AS next_id FROM $(table_name)"
df_next_id = Wds.sql_df(sql)

,next_id
,Int64?
1,348537


In [66]:
next_id = df_next_id[1, :next_id] + 1

348538

In [67]:
df_prp_r[!, :id] .= collect(next_id:(next_id + size(df_prp_r, 1) - 1));

In [68]:
df_prp_r[1000:1007, [:id, :person_id, :ID_Domherr, :Amtsart]]

,id,person_id,ID_Domherr,Amtsart
,Int64,Int32?,Int32,String?
1,349537,130139,17835,Domherr
2,349538,130135,17839,Domherr
3,349539,123067,10368,Domherr
4,349540,130620,18314,Domherr
5,349541,130610,18324,Domherr
6,349542,130652,18342,Domherr
7,349543,122711,10004,Domherr
8,349544,122802,10093,Domherr


In [69]:
columns = [
    :id => :id,
    :Bistum => :diocese_name,
    :Amtsbeginn => :date_begin,
    :Amtsende => :date_end,
    :Amt_Kommentar => :note,
    :Sort => :display_order,
    :Amtsart => :role_name,
    :person_id => :person_id,
    :diocese_id => :diocese_id,
    :role_id => :role_id,
    :institution_id => :institution_id,
    :institution_name => :institution_name,
    :num_date_begin => :num_date_begin,
    :num_date_end => :num_date_end,
    :date_sort_key => :date_sort_key,
];

In [70]:
maximum(df_prp_r.date_sort_key)

9000900

In [71]:
table_name = "person_role"
Wds.filltable!(table_name, select(df_prp_r, columns))

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1185
┌ Info: 20000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1185
┌ Info: 30000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1185
┌ Info: Rows inserted: 32988
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


32988

Titularbistum ist noch nicht eingetragen

## Zeitspanne der Person

In [191]:
table_name = "person"
sql = "SELECT id, num_date_birth, num_date_death
FROM $(table_name) WHERE id in (SELECT id FROM item WHERE item_type_id = $(item_type_id))"
df_date = Wds.sql_df(sql);

In [192]:
size(df_date)

(22729, 3)

Zeitspanne der Ämter

In [194]:
df_pr_begin_0 = dropmissing(df_prp_r, :num_date_begin)
df_pr_begin = groupby(select(df_pr_begin_0, :person_id, :num_date_begin), :person_id)

,person_id,num_date_begin
,Int32?,Int64
1,122686,1680
2,122686,1678
3,122686,1677
,person_id,num_date_begin
,Int32?,Int64
1,145465,1686
2,145465,1663
3,145465,1663
4,145465,1667


In [195]:
df_pr_date_min = combine(df_pr_begin, :num_date_begin => minimum => :date_min);

In [196]:
size(df_pr_date_min)

(22018, 2)

In [197]:
df_pr_date_min[3015:3021, :]

,person_id,date_min
,Int32?,Int64
1,125728,1439
2,125729,1436
3,125730,1428
4,125731,1387
5,125732,1434
6,125733,1432
7,125734,1409


`date_max`

In [198]:
df_pr_end_value = dropmissing(df_prp_r, :num_date_end)
df_pr_end = groupby(select(df_pr_end_value, :person_id, :num_date_end), :person_id)

,person_id,num_date_end
,Int32?,Int64
1,122686,1690
2,122686,1690
3,122686,1690
,person_id,num_date_end
,Int32?,Int64
1,145465,1688


In [199]:
df_pr_date_max = combine(df_pr_end, :num_date_end => maximum => :date_max);

In [200]:
size(df_pr_date_max)

(15595, 2)

In [201]:
df_pr_date_max[3015:3021, :]

,person_id,date_max
,Int32?,Int64
1,126648,1368
2,126651,1391
3,126652,1368
4,126653,1389
5,126655,1368
6,126657,1381
7,126658,1377


In [202]:
df_pr_date_min_mng = subset(df_pr_date_min, :person_id => ByRow(ismissing));

In [203]:
size(df_pr_date_min_mng)

(0, 2)

In [204]:
subset!(df_pr_date_min, :person_id => ByRow(!ismissing));

In [205]:
df_date = leftjoin(df_date, df_pr_date_min, on = :id => :person_id);

In [206]:
subset!(df_pr_date_max, :person_id => ByRow(!ismissing));

In [207]:
df_date = leftjoin(df_date, df_pr_date_max, on = :id => :person_id);

Geburts- und Sterbedatum  
Geburts- und Sterbedaten - so vorhanden - können die Zeitspanne nur verkürzen!

In [208]:
min_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : min(x, y));
max_may_be(x, y) = ismissing(x) ? y : (ismissing(y) ? x : max(x, y));

In [209]:
transform!(df_date, [:date_min, :num_date_birth] => ByRow(max_may_be) => :date_min); # sic!

In [210]:
transform!(df_date, [:date_max, :num_date_death] => ByRow(min_may_be) => :date_max); # sic!

Übernimm' den jeweils gültigen Wert für fehlende Daten

In [211]:
get_first_not_missing(x, y) = !ismissing(x) ? x : y;

In [212]:
transform!(df_date, [:date_min, :date_max] => ByRow(get_first_not_missing) => :date_min);

In [213]:
transform!(df_date, [:date_max, :date_min] => ByRow(get_first_not_missing) => :date_max);

Damit sind Einträge mit fehlenden Daten entbehrlich

In [214]:
dropmissing!(df_date, :date_min);

Trage die geänderten Daten ein

In [215]:
sql = "CREATE TEMPORARY TABLE date_update (id INT, date_min INT, date_max INT) 
ENGINE=InnoDB DEFAULT CHARSET=utf8;";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "CREATE TEMPORARY TABLE date_update (id INT, date_min INT, date_max INT) \nENGINE=InnoDB DEFAULT CHARSET=utf8;", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [216]:
names(df_date)

5-element Vector{String}:
 "id"
 "num_date_birth"
 "num_date_death"
 "date_min"
 "date_max"

In [217]:
columns = [
    :id => :id,
    :date_min => :date_min,
    :date_max => :date_max
]

3-element Vector{Pair{Symbol, Symbol}}:
       :id => :id
 :date_min => :date_min
 :date_max => :date_max

In [218]:
table_name = "date_update"
Wds.filltable!(table_name, select(df_date, columns));

┌ Info: 10000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1185
┌ Info: 20000
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1185
┌ Info: Rows inserted: 22104
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


In [219]:
sql = "UPDATE person AS p, 
(SELECT id, date_min, date_max FROM date_update) AS du
SET p.date_min = du.date_min, p.date_max = du.date_max
WHERE p.id = du.id";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "UPDATE person AS p, \n(SELECT id, date_min, date_max FROM date_update) AS du\nSET p.date_min = du.date_min, p.date_max = du.date_max\nWHERE p.id = du.id", 0, -1, 0, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

*2022-01-17* Es gibt etliche Einträge für Personen mit Ämtern nach 2000?! 
```sql
select p.id, id_in_source, item_type_id, givenname, familyname, date_min, date_max, edit_status, is_online
from person as p join item on item.id = p.id and p.date_max > 2000;
```

## Archidiakonat

In [72]:
println.(names(df_prp_r));

Bistum
Amtsart
Amtsbeginn
Amtsende
ID_Amt
Institution
Amt_weltlich
Herrschaftsgebiet
Ort
Amt_Kommentar
Sort
Titularbistum
Profession
ID_Kloster
ID_Fremdschlüssel
Dignität
ID_Domherr
Archidiakonat
Hilfsfeld_import
ID_Amt_norm
id_in_source
person_id
match_role_id
role_name
role_id
diocese_id
institution_id
institution_name
num_date_begin
num_date_end
date_sort_key
date_sort_key_replace
id


In [73]:
columns = [
    :id => :person_role_id,
    :person_id => :person_id,
    :Archidiakonat => :value
]

3-element Vector{Pair{Symbol, Symbol}}:
            :id => :person_role_id
     :person_id => :person_id
 :Archidiakonat => :value

In [74]:
df_pr_property = select(df_prp_r, columns);

In [75]:
dropmissing!(df_pr_property, :value);

In [76]:
size(df_pr_property)

(464, 3)

In [77]:
df_pr_property[!, :name] .= "Archidiakonat";

In [78]:
df_pr_property[407:414, :]

,person_role_id,person_id,value,name
,Int64,Int32?,String,String
1,366150,132729,Sarstedt,Archidiakonat
2,366162,132723,1359,Archidiakonat
3,366164,132722,1489,Archidiakonat
4,366166,132722,ca. 1520,Archidiakonat
5,366169,132720,Pattensen,Archidiakonat
6,366230,132761,Goslar,Archidiakonat
7,366231,132761,Bockenem,Archidiakonat
8,366233,132760,Goslar,Archidiakonat


In [83]:
table_name = "person_role_property";
sql = "DELETE FROM $(table_name)
WHERE person_id IN
(SELECT id FROM item WHERE item_type_id = $(item_type_id))";
DBInterface.execute(Wds.dbwiag, sql)

MySQL.TextCursor{true}(MySQL.Connection(host="127.0.0.1", user="georg", port="3306", db="wiag2"), "DELETE FROM person_role_property\nWHERE person_id IN\n(SELECT id FROM item WHERE item_type_id = 5)", 0, -1, 928, MySQL.API.MYSQL_RES(Ptr{Nothing} @0x0000000000000000), Symbol[], Type[], Dict{Symbol, Int64}(), 0, 1, false)

In [84]:
table_name = "person_role_property"
Wds.filltable!(table_name, df_pr_property)

┌ Info: Rows inserted: 464
└ @ WiagDataSetup C:\Users\georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1208


464

## Sortierung der Person nach Amt
Die Tabelle `person_display_order` ist eine technische Hilfstabelle, um die Abfragen in Doctrine zu vereinfachen oder überhaupt erst zu ermöglichen. Jede Person erhält für jedes Bistum einen Sortierschlüssel und einen weiteren unabhängig vom Bistum.

*2022-01-17* Verzichte zunächst auf diese Tabelle und löse das Problem in der Web-Anwendung. So bleibt das Datenmodell schlanker.

In [45]:
using MySQL, DataFrames

Sortierung unabhängig von einer Diözese. Es werden auch Ämter berücksichtigt, die nicht mit einer Diözese verbunden sind.

In [48]:
sql = "SELECT person_id, 'any' as diocese, min(display_order) as display_order " *
"FROM person_role GROUP BY person_id";
df_do = db_exec(sql);

In [49]:
size(df_do)

(5159, 3)

Zwölf Bischöfe haben kein Amt!?

In [50]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 5159
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


5159

Sortierung nach Diözese

In [52]:
sql = "SELECT person_id, diocese_name as diocese, min(display_order) as display_order " *
"FROM person_role " *
"WHERE diocese_name IS NOT NULL " *
"GROUP BY person_id, diocese_name";
df_do = db_exec(sql);

In [53]:
size(df_do)

(6370, 3)

In [54]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do)

┌ Info: Rows inserted: 6370
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


6370

Bischöfe ohne Ämter

In [55]:
sql = "SELECT id AS person_id, 'any' as diocese, 950000 AS display_order " *
"FROM person WHERE id NOT IN (SELECT DISTINCT person_id FROM person_role)"
df_do_r = db_exec(sql);

In [56]:
size(df_do_r)

(12, 3)

In [57]:
table_dst = "person_display_order"
Wds.filltable!(table_dst, df_do_r)

┌ Info: Rows inserted: 12
└ @ WiagDataSetup C:\Users\Georg\Documents\projekte\WiagDataSetup.jl\src\WiagDataSetup.jl:1169


12